<a href="https://colab.research.google.com/github/Tonry12/Machine-learning-Advance/blob/main/video_to_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import missingno as msno
from google.colab import drive
drive.mount('/content/drive')
# path = "/content/drive/MyDrive/smarticu_data/work_files/"

Mounted at /content/drive


In [2]:
pip install pytesseract

In [5]:
# ติดตั้ง Tesseract ใน Colab
!apt-get install -y tesseract-ocr

# ติดตั้ง pytesseract
!pip install pytesseract

# กำหนด path ของ Tesseract
import pytesseract
from PIL import Image
import os



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (12.0 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [7]:
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'  # Path ของ Tesseract ที่ติดตั้งใน Colab

Text from Annotation 2024-12-09 212042.png:
Use Cloud Vision API and AutoML to tag and
process images

Image tagging is also referred to as image labeling.

Cloud Vision API can identify and label general objects,
landmarks, locations, logos, activities, animal species,
products, and more in an image. Once the images are tagged
with the detected labels, image search, processing and
management are automated and easier.

If you need targeted custom labels, use Cloud AutoML to train
a custom ML model.

To use Google OCR technologies on premise, use OCR On-
Prem, available in the Cloud Marketplace.

--------------------------------------------------


In [9]:
pip install opencv-python


In [13]:
import cv2
import os

# ฟังก์ชันสำหรับจับภาพจากวิดีโอ
def save_frames_from_video(video_path, output_folder):
    # เปิดไฟล์วิดีโอ
    cap = cv2.VideoCapture(video_path)

    # ตรวจสอบว่าการเปิดไฟล์วิดีโอสำเร็จหรือไม่
    if not cap.isOpened():
        print("ไม่สามารถเปิดไฟล์วิดีโอได้")
        return

    # สร้างโฟลเดอร์สำหรับเก็บภาพถ้าหากยังไม่มี
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    frame_count = 0
    while True:
        # อ่านเฟรมจากวิดีโอ
        ret, frame = cap.read()

        # ถ้าไม่สามารถอ่านเฟรมได้ (ถึงจุดสิ้นสุดของวิดีโอ)
        if not ret:
            break

        # สร้างชื่อไฟล์สำหรับบันทึกภาพ (ใช้ตัวเลขลำดับ)
        frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")

        # บันทึกภาพเป็นไฟล์ .jpg
        cv2.imwrite(frame_filename, frame)

        # เพิ่มตัวนับเฟรม
        frame_count += 1

    # ปิดการเชื่อมต่อกับวิดีโอ
    cap.release()
    print(f"บันทึกภาพทั้งหมด {frame_count} ภาพในโฟลเดอร์ {output_folder}")

# ตัวอย่างการใช้งาน
video_path = '/content/drive/MyDrive/Video for python/ScreenRecording_11-29-2567 09-12-16_1.mov'  # แก้ไขเป็น path ของไฟล์วิดีโอ
output_folder = 'form_video/frames'  # โฟลเดอร์ที่ต้องการให้บันทึกภาพ

save_frames_from_video(video_path, output_folder)


บันทึกภาพทั้งหมด 11514 ภาพในโฟลเดอร์ form_video/frames


In [17]:
import cv2
import os
from skimage.metrics import structural_similarity as ssim
import numpy as np

# ฟังก์ชันสำหรับคำนวณ MSE
def mse(imageA, imageB):
    # คำนวณค่า Mean Squared Error (MSE) ระหว่างภาพสองภาพ
    return np.sum((imageA - imageB) ** 2) / float(imageA.shape[0] * imageA.shape[1])

# ฟังก์ชันสำหรับจับภาพจากวิดีโอและลบภาพซ้ำ
def save_frames_from_video(video_path, output_folder, threshold_ssim=0.99, threshold_mse=1000):
    # เปิดไฟล์วิดีโอ
    cap = cv2.VideoCapture(video_path)

    # ตรวจสอบว่าการเปิดไฟล์วิดีโอสำเร็จหรือไม่
    if not cap.isOpened():
        print(f"ไม่สามารถเปิดไฟล์วิดีโอ: {video_path}")
        return

    # สร้างโฟลเดอร์สำหรับเก็บภาพตามชื่อของไฟล์วิดีโอ
    video_filename = os.path.basename(video_path)
    video_name, _ = os.path.splitext(video_filename)  # เอาชื่อไฟล์โดยไม่เอานามสกุล
    video_output_folder = os.path.join(output_folder, video_name)

    if not os.path.exists(video_output_folder):
        os.makedirs(video_output_folder)

    prev_frame = None  # ตัวแปรเก็บเฟรมก่อนหน้า
    frame_count = 0

    while True:
        # อ่านเฟรมจากวิดีโอ
        ret, frame = cap.read()

        # ถ้าไม่สามารถอ่านเฟรมได้ (ถึงจุดสิ้นสุดของวิดีโอ)
        if not ret:
            break

        # แปลงภาพเป็นขาวดำเพื่อทำการเปรียบเทียบได้ง่ายขึ้น
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # เปรียบเทียบภาพกับภาพก่อนหน้า
        if prev_frame is not None:
            # คำนวณ SSIM และ MSE
            ssim_value = ssim(prev_frame, gray_frame)
            mse_value = mse(prev_frame, gray_frame)

            # ถ้าภาพมีความคล้ายคลึงกันเกิน threshold (เช่น 0.99 สำหรับ SSIM หรือ MSE < 1000)
            if ssim_value > threshold_ssim or mse_value < threshold_mse:
                print(f"ภาพที่ {frame_count} คล้ายกับภาพก่อนหน้า จึงข้ามไป")
                continue

        # สร้างชื่อไฟล์สำหรับบันทึกภาพ (ใช้ตัวเลขลำดับ)
        frame_filename = os.path.join(video_output_folder, f"frame_{frame_count}.jpg")

        # บันทึกภาพเป็นไฟล์ .jpg
        cv2.imwrite(frame_filename, frame)

        # เพิ่มตัวนับเฟรม
        prev_frame = gray_frame
        frame_count += 1

    # ปิดการเชื่อมต่อกับวิดีโอ
    cap.release()
    print(f"บันทึกภาพทั้งหมด {frame_count} ภาพในโฟลเดอร์ {video_output_folder}")

# ตัวอย่างการใช้งาน
video_path = '/content/drive/MyDrive/Video for python/ScreenRecording_11-29-2567 09-12-16_1.mov'  # แก้ไขเป็น path ของไฟล์วิดีโอ
output_folder = 'form_video/frames'  # แก้ไขเป็น path ของโฟลเดอร์ที่ต้องการบันทึกภาพ
save_frames_from_video(video_path, output_folder)


ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนหน้า จึงข้ามไป
ภาพที่ 1 คล้ายกับภาพก่อนห

KeyboardInterrupt: 

In [18]:
import cv2
import os
import numpy as np

# ฟังก์ชันเปรียบเทียบ histogram ของภาพ
def are_histograms_similar(image1, image2, threshold=0.9):
    # แปลงภาพเป็น grayscale
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # คำนวณ histogram ของภาพทั้งสอง
    hist1 = cv2.calcHist([image1_gray], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([image2_gray], [0], None, [256], [0, 256])

    # คำนวณการเปรียบเทียบ histogram โดยใช้ correlation
    correlation = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

    # ถ้าค่า correlation มากกว่า threshold แสดงว่าเป็นภาพซ้ำ
    if correlation > threshold:
        return True
    else:
        return False

# ฟังก์ชันสำหรับการบันทึกภาพที่ไม่ซ้ำ
def save_unique_frames(images, output_folder, threshold=0.9):
    # สร้างโฟลเดอร์สำหรับเก็บภาพที่ไม่ซ้ำ
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    saved_images = []
    image_count = 0

    for image in images:
        is_similar = False
        for saved_image in saved_images:
            if are_histograms_similar(image, saved_image, threshold):
                is_similar = True
                break

        if not is_similar:
            # ถ้าภาพไม่ซ้ำ ก็เก็บบันทึกภาพนั้น
            image_filename = os.path.join(output_folder, f"frame_{image_count}.jpg")
            cv2.imwrite(image_filename, image)
            saved_images.append(image)
            image_count += 1
            print(f"บันทึกภาพ {image_filename}")

    print(f"บันทึกภาพทั้งหมด {image_count} ภาพในโฟลเดอร์ {output_folder}")

# ตัวอย่างการใช้งาน
images = []  # สมมติว่าเรามีรายการของภาพที่ต้องการตรวจสอบ
# บางทีคุณอาจจะเปิดภาพจากไฟล์หรือจับภาพจากวิดีโอ

output_folder = "/content/form_video/frames/unique_images"
save_unique_frames(images, output_folder)


บันทึกภาพทั้งหมด 0 ภาพในโฟลเดอร์ /content/form_video/frames/unique_images


In [ ]:
import cv2
import os

# ฟังก์ชันสำหรับจับภาพจากวิดีโอ
def save_frames_from_video(video_path, output_folder):
    # เปิดไฟล์วิดีโอ
    cap = cv2.VideoCapture(video_path)

    # ตรวจสอบว่าการเปิดไฟล์วิดีโอสำเร็จหรือไม่
    if not cap.isOpened():
        print(f"ไม่สามารถเปิดไฟล์วิดีโอ: {video_path}")
        return

    # สร้างโฟลเดอร์สำหรับเก็บภาพตามชื่อของไฟล์วิดีโอ
    video_filename = os.path.basename(video_path)
    video_name, _ = os.path.splitext(video_filename)  # เอาชื่อไฟล์โดยไม่เอานามสกุล
    video_output_folder = os.path.join(output_folder, video_name)

    if not os.path.exists(video_output_folder):
        os.makedirs(video_output_folder)

    frame_count = 0
    while True:
        # อ่านเฟรมจากวิดีโอ
        ret, frame = cap.read()

        # ถ้าไม่สามารถอ่านเฟรมได้ (ถึงจุดสิ้นสุดของวิดีโอ)
        if not ret:
            break

        # สร้างชื่อไฟล์สำหรับบันทึกภาพ (ใช้ตัวเลขลำดับ)
        frame_filename = os.path.join(video_output_folder, f"frame_{frame_count}.jpg")

        # บันทึกภาพเป็นไฟล์ .jpg
        cv2.imwrite(frame_filename, frame)

        # เพิ่มตัวนับเฟรม
        frame_count += 1

    # ปิดการเชื่อมต่อกับวิดีโอ
    cap.release()
    print(f"บันทึกภาพทั้งหมด {frame_count} ภาพในโฟลเดอร์ {video_output_folder}")

# ฟังก์ชันสำหรับจับภาพจากหลายไฟล์วิดีโอ
def save_frames_from_multiple_videos(videos_list, output_folder):
    for video_path in videos_list:
        save_frames_from_video(video_path, output_folder)

# ตัวอย่างการใช้งาน
videos_list = [
    'form_video/video/video1.mp4',
    'form_video/video/video2.mp4',
    'form_video/video/video3.mp4'
]  # ลิสต์ของไฟล์วิดีโอ

output_folder = './output_frames'  # โฟลเดอร์หลักที่จะเก็บภาพทั้งหมด

save_frames_from_multiple_videos(videos_list, output_folder)


In [19]:
from PIL import Image
import pytesseract
import os

# กำหนด path ที่ติดตั้ง Tesseract (สำหรับ Windows อาจจะต้องระบุ path ที่ถูกต้อง)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # แก้ไขตามตำแหน่งที่ติดตั้ง

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# ฟังก์ชันหลักในการแปลงข้อมูลจากหลายไฟล์ภาพ
def extract_text_from_images_in_folder(folder_path):
    text_data = {}

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)
            text_data[filename] = extracted_text

    return text_data

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขให้เป็น path ของโฟลเดอร์ที่เก็บภาพ
extracted_text = extract_text_from_images_in_folder(folder_path)

# แสดงข้อมูลที่ดึงออกมา
for filename, text in extracted_text.items():
    print(f"Text from {filename}:")
    print(text)
    print("-" * 50)


Text from frame_10041.jpg:
sau 709
Pra PA0S
PrN A0g
PNT WA0S
TTA 0)S
PT WA0y

Posty 703

00:05.72

00:04.41

00:02.76

00:01.50

00:03.21

ole ev ara

00:03.03

--------------------------------------------------
Text from frame_0.jpg:
Pert,
Pea
Pry es
Phi
Pry Re
Prt

sau 1

00:02.11

00:03.44

00:05.56

olen er. oye!

00:09.69

00:21.46

00:23.41

--------------------------------------------------


In [20]:
from PIL import Image
import pytesseract
import os
import cv2
import numpy as np

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ (ปรับภาพให้ชัดเจนขึ้น)
def extract_text_from_image(image_path):
    # เปิดภาพด้วย PIL
    img = Image.open(image_path)

    # แปลงเป็น grayscale ด้วย OpenCV
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    gray_img = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    # ปรับคอนทราสต์เพื่อให้ภาพคมชัดขึ้น (ใช้ thresholding)
    _, thresh_img = cv2.threshold(gray_img, 150, 255, cv2.THRESH_BINARY)

    # แปลงกลับเป็น PIL Image เพื่อให้ Tesseract ใช้งานได้
    pil_img = Image.fromarray(thresh_img)

    # ใช้ Tesseract OCR เพื่อดึงข้อความ
    text = pytesseract.image_to_string(pil_img, config='--psm 6')  # PSM 6 ใช้สำหรับการตรวจจับข้อความในบรรทัดเดียว

    return text

# ฟังก์ชันหลักในการแปลงข้อมูลจากหลายไฟล์ภาพ
def extract_text_from_images_in_folder(folder_path):
    text_data = {}

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)
            text_data[filename] = extracted_text

    return text_data

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขให้เป็น path ของโฟลเดอร์ที่เก็บภาพ
extracted_text = extract_text_from_images_in_folder(folder_path)

# แสดงข้อมูลที่ดึงออกมา
for filename, text in extracted_text.items():
    print(f"Text from {filename}:")
    print(text)
    print("-" * 50)


Text from frame_10041.jpg:
PTT AO 00:05.72
PTT PA0S OHOY wa
Pry WA0YA 00:02.76
PTT A0S 00:01.50
PTT AOS OHO wa
PTT A0y 00:07.71
PTT WAOk 00:03.03

--------------------------------------------------
Text from frame_0.jpg:
er OHO ai
Pra) 00:03.44
saun 5 00:05.56
are 00:04.54
Pri) 00:09.69
ai 00:21.46
ere eee el

--------------------------------------------------


In [21]:
import pandas as pd
import re
from PIL import Image
import pytesseract
import os

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# ฟังก์ชันหลักในการแปลงข้อมูลจากหลายไฟล์ภาพ
def extract_times_from_images_in_folder(folder_path):
    time_data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regular expression เพื่อดึงเวลา
            times = re.findall(r'\d{2}:\d{2}\.\d{2}', extracted_text)  # รูปแบบเวลา 00:00.00
            time_data.extend(times)  # เพิ่มผลลัพธ์ลงใน time_data

    return time_data

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขให้เป็น path ของโฟลเดอร์ที่เก็บภาพ
times = extract_times_from_images_in_folder(folder_path)

# แสดงผลใน DataFrame
df = pd.DataFrame(times, columns=["Time"])
df


,Time
0,00:05.72
1,00:04.41
2,00:02.76
3,00:01.50
4,00:03.21
5,00:03.03
6,00:02.11
7,00:03.44
8,00:05.56
9,00:09.69


In [22]:
import pandas as pd
import re
from PIL import Image
import pytesseract
import os

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regex เพื่อดึง "รอบที่" และเวลา
            rounds = re.findall(r'รอบที่(\d+)', extracted_text)
            times = re.findall(r'\d{2}:\d{2}\.\d{2}', extracted_text)

            # ถ้ามีทั้งรอบและเวลาให้เก็บข้อมูลลงใน data
            for round_num, time in zip(rounds, times):
                data.append({"รอบ": round_num, "เวลา": time})

    # สร้าง DataFrame จากข้อมูลที่ดึงออกมา
    df = pd.DataFrame(data)

    return df

# ตัวอย่างการใช้งาน
folder_path = 'แ'  # แก้ไขให้เป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์ใน DataFrame
df


""


In [23]:
import pandas as pd
import re
from PIL import Image
import pytesseract
import os

# กำหนด path ที่ติดตั้ง Tesseract (กรณี Windows อาจจะต้องระบุ path ที่ถูกต้อง)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # หรือระบุ path ที่ถูกต้อง

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพและใช้ภาษาไทย
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    # ใช้ 'tha' สำหรับภาษาไทย
    text = pytesseract.image_to_string(img, lang='tha')
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regular expression เพื่อดึงข้อมูลรอบและเวลา
            rounds_and_times = re.findall(r'รอบที่\s*(\d+).*?(\d{2}:\d{2}\.\d{2})', extracted_text)

            # ถ้ามีผลลัพธ์จาก regex, เก็บลงใน data
            for round_num, time in rounds_and_times:
                data.append([round_num, time])

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data, columns=["รอบที่", "เวลา"])
    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขเป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์
print(df)


TesseractError: (1, 'Error opening data file /usr/share/tesseract-ocr/4.00/tessdata/tha.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'tha\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [24]:
# ติดตั้ง Tesseract และไฟล์ภาษาไทย
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-tha

# ตรวจสอบว่า Tesseract ติดตั้งเรียบร้อยแล้ว
!tesseract --version


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,225 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Hit:13 h

In [25]:
import pytesseract
from PIL import Image
import os
import pandas as pd
import re

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='tha')  # ใช้ภาษาไทย
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regular expression เพื่อดึงตัวเลขรอบและเวลา
            rounds_and_times = re.findall(r'รอบที่(\d+).*?(\d{2}:\d{2}\.\d{2})', extracted_text)

            # เพิ่มข้อมูลลงในรายการ
            for round_number, time in rounds_and_times:
                data.append([round_number, time])

    # แปลงข้อมูลเป็น DataFrame
    df = pd.DataFrame(data, columns=['Round', 'Time'])
    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขเป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์
df


,Round,Time


In [ ]:
# ติดตั้ง Tesseract และไฟล์ภาษาไทย
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-tha

# ตรวจสอบการติดตั้ง Tesseract
!tesseract --version
!ls /usr/share/tesseract-ocr/4.00/tessdata/


In [45]:

import pytesseract
from PIL import Image
import os
import pandas as pd
import re

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='tha')  # ใช้ภาษาไทย
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # ตรวจสอบว่าโฟลเดอร์มีไฟล์หรือไม่
    if not os.path.exists(folder_path):
        print("ไม่พบโฟลเดอร์ที่กำหนด")
        return data

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            print(f"กำลังประมวลผลไฟล์: {filename}")

            extracted_text = extract_text_from_image(file_path)

            # แสดงข้อความที่ดึงออกมา
            print(f"ข้อความจาก {filename}:")
            print(extracted_text)

            # ใช้ regular expression เพื่อดึงเวลาที่เป็นรูปแบบ 00:21.46 และรอบที่
            round_and_times = re.findall(r'รอบที่\s*(\d+).*?(\d{2}:\d{2}\.\d{2})', extracted_text)
            for round_num, time in round_and_times:
                data.append({"รอบที่": round_num, "เวลา": time})

    return data

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # เปลี่ยนเป็น path ของคุณ
data = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์ในรูปแบบ DataFrame
if data:
    df = pd.DataFrame(data)
    print(df)
else:
    print("ไม่พบข้อมูลที่สามารถดึงออกมาได้")


กำลังประมวลผลไฟล์: frame_10041.jpg
ข้อความจาก frame_10041.jpg:
รอบที่ 709
รอบที่ 708
รอบที่ 707
รอบที่ 706
เว 11จะ
รอบที่ 704

0 703

00:05.72

เง ง

00:02.76

00:01.50

00:03.21

7

00:03.03

กำลังประมวลผลไฟล์: frame_0.jpg
ข้อความจาก frame_0.jpg:
1
รอบที่ 6
เกา11๓
เกา1
11
1 9

ว1

ง4

00:03.44

01อ05รได

00:04.54

00:09.69

00:21.46

00:23.41

ไม่พบข้อมูลที่สามารถดึงออกมาได้


In [46]:
import pytesseract
from PIL import Image
import cv2
import os
import pandas as pd
import re
import numpy as np

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับเพิ่มความคมชัดของภาพ
def enhance_image(image_path):
    # อ่านภาพจากไฟล์
    img = cv2.imread(image_path)

    # แปลงภาพเป็นขาวดำ (Grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ใช้ฟิลเตอร์ Sharpening เพื่อเพิ่มความคมชัด
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    sharpened = cv2.filter2D(gray, -1, kernel)

    # บันทึกภาพที่คมชัดขึ้น
    sharpened_image_path = 'enhanced_image.jpg'
    cv2.imwrite(sharpened_image_path, sharpened)

    return sharpened_image_path

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    # ใช้ภาษาไทยสำหรับการแยกคำว่า "รอบที่"
    text = pytesseract.image_to_string(img, lang='tha')
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)

            # เพิ่มความคมชัดให้ภาพ
            enhanced_image_path = enhance_image(file_path)

            # ดึงข้อความจากภาพที่คมชัดขึ้น
            extracted_text = extract_text_from_image(enhanced_image_path)

            # ใช้ regex เพื่อดึงข้อมูลเฉพาะ "รอบที่" และ "เวลา"
            matches = re.findall(r'รอบที่\s*(\d+)\s*(\d{2}:\d{2}\.\d{2})', extracted_text)

            # หากพบการจับคู่ให้เก็บลงใน data
            for match in matches:
                round_number = match[0]  # หมายเลขรอบ
                time = match[1]  # เวลา
                data.append([round_number, time])

    # แปลงข้อมูลเป็น DataFrame
    df = pd.DataFrame(data, columns=['รอบที่', 'เวลา'])
    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขให้เป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์
print(df)


  รอบที่      เวลา
0    708  00:04.41
1    704  00:07.71


In [44]:
import pytesseract
from PIL import Image
import os
import pandas as pd
import re

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='tha')  # ใช้ภาษาไทย
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regex เพื่อดึงตัวเลขรอบและเวลา
            rounds = re.findall(r'รอบที่ (\d+)', extracted_text)  # ดึงตัวเลขรอบ
            times = re.findall(r'\d{2}:\d{2}.\d{2}', extracted_text)  # ดึงเวลาที่มีรูปแบบ 00:21.46

            # รวมข้อมูลเป็นคู่ระหว่างรอบและเวลา
            for round_num, time in zip(rounds, times):
                data.append([round_num, time])

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data, columns=['รอบที่', 'เวลา'])
    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขเป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์
print(df)


  รอบที่      เวลา
0    709  00:05.72
1    708  00:02.76
2    707  00:01.50
3    706  00:03.21
4    704  00:03.03
5      6  00:03.44


In [47]:
import pytesseract
from PIL import Image
import os
import pandas as pd
import re

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='tha')  # ใช้ภาษาไทย
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regex เพื่อดึงตัวเลขรอบและเวลา
            rounds = re.findall(r'รอบที่ (\d+)', extracted_text)  # ดึงตัวเลขรอบ
            times = re.findall(r'\d{2}:\d{2}.\d{2}', extracted_text)  # ดึงเวลาที่มีรูปแบบ 00:21.46

            # รวมข้อมูลเป็นคู่ระหว่างรอบและเวลา
            for round_num, time in zip(rounds, times):
                data.append([round_num, time])

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data, columns=["รอบที่", "เวลา"])

    # ลบแถวที่ซ้ำ
    df = df.drop_duplicates()

    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขให้เป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผล DataFrame ที่ไม่มีค่าซ้ำ
df


,รอบที่,เวลา
0,6,00:02.11
1,709,00:05.72
2,708,00:02.76
3,707,00:01.50
4,706,00:03.21
5,704,00:03.03
6,6,00:03.44


In [43]:
import pytesseract
from PIL import Image
import cv2
import os
import pandas as pd
import re

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับเพิ่มความคมชัดของภาพ
def enhance_image(image_path):
    # อ่านภาพจากไฟล์
    img = cv2.imread(image_path)

    # แปลงภาพเป็นขาวดำ (Grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ใช้ฟิลเตอร์ Sharpening เพื่อเพิ่มความคมชัด
    kernel = np.array([[-1, -1, -1],
                       [-1,  9, -1],
                       [-1, -1, -1]])
    sharp_img = cv2.filter2D(gray, -1, kernel)

    # ใช้การปรับคอนทราสต์
    contrast_img = cv2.convertScaleAbs(sharp_img, alpha=1.5, beta=0)  # alpha: คอนทราสต์, beta: ความสว่าง

    return contrast_img

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพที่ผ่านการปรับความคมชัด
def extract_text_from_image(image_path):
    img = enhance_image(image_path)  # เพิ่มความคมชัดของภาพ
    # แปลงภาพที่คมชัดแล้วเป็น Image ใน PIL
    pil_img = Image.fromarray(img)
    text = pytesseract.image_to_string(pil_img, lang='tha')  # ใช้ภาษาไทย
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regex เพื่อดึงตัวเลขรอบและเวลา
            rounds_and_times = re.findall(r'รอบที่\s*(\d+).*\s*(\d{2}:\d{2}\.\d{2})', extracted_text)
            for round_num, time in rounds_and_times:
                data.append([round_num, time])

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data, columns=["รอบที่", "เวลา"])
    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # แก้ไขเป็น path ของโฟลเดอร์ที่เก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผล
print(df)


  รอบที่      เวลา
0    708  00:04.41
1    704  00:07.71
2      6  00:03.44
3      5  00:05.56


In [30]:
import pytesseract
from PIL import Image
import os
import pandas as pd
import re

# กำหนดให้ Tesseract ใช้ภาษาไทย
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# ฟังก์ชันสำหรับอ่านข้อมูลจากภาพ
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='tha')  # ใช้ภาษาไทย
    return text

# ฟังก์ชันหลักในการดึงข้อมูลจากหลายไฟล์ภาพ
def extract_round_and_times_from_images_in_folder(folder_path):
    data = []

    # วนลูปในทุกไฟล์ในโฟลเดอร์ที่กำหนด
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # กรองไฟล์ที่เป็นภาพ
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)

            # ใช้ regex เพื่อดึงตัวเลขรอบและเวลา
            # ใช้ regex ที่สามารถดึงทั้งคำว่า "รอบที่" และเวลาที่เป็นรูปแบบ HH:MM:SS.xx
            rounds_times = re.findall(r'รอบที่\s*(\d+)[^0-9]+(\d{2}:\d{2}\.\d{2})', extracted_text)

            # เพิ่มข้อมูลเข้าไปใน list
            for round_num, time in rounds_times:
                data.append([round_num, time])

    # แปลงข้อมูลเป็น DataFrame
    df = pd.DataFrame(data, columns=["รอบที่", "เวลา"])
    return df

# ตัวอย่างการใช้งาน
folder_path = '/content/form_video/frames/ScreenRecording_11-29-2567 09-12-16_1'  # ปรับ path ให้ตรงกับที่คุณเก็บภาพ
df = extract_round_and_times_from_images_in_folder(folder_path)

# แสดงผลลัพธ์
df


,รอบที่,เวลา
